# import packages

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import gzip
import shutil
import os

C:\Users\monkeydc\.conda\envs\561\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# data processing

In [2]:
def word_type(word:str):
    if word.isdigit():
        return "unk_num"
    elif word.islower(): 
        return "unk_all_lower"    
    elif word.isupper(): 
        return "unk_all_upper"              
    elif any(char.isdigit() for char in word):
        return "unk_contain_num"    
    else:
        return "unk"

In [3]:
def read_in_data(filename):
    vocab = dict()
    sentences =[] # generating original output
    tags = set()
    tmp_sentence = []
    targets = []
    tmp_target = []
    with open(filename, "r") as file:
        for line in file.readlines():
            if len(line) > 1:
                _, word, tag = line.strip().split(" ")
                if word not in vocab.keys():
                    vocab[word] = 1
                else:
                    vocab[word] += 1
                tags.add(tag)
                tmp_sentence.append(word)
                tmp_target.append(tag)
            else:
                sentences.append(tmp_sentence)
                targets.append(tmp_target)
                tmp_sentence = []
                tmp_target = []
        if len(tmp_sentence)>0:
            sentences.append(tmp_sentence)
            targets.append(tmp_target)
    return vocab, sentences, tags, targets

def remove_low_frequency_word(vocab, occurences):
    candidates = set()
    for word in vocab.keys():
        if vocab[word] >= occurences:
            if any(w.isdigit() for w in word):
                candidates.add(word_type(word))
            else:
                candidates.add(word)
        else:
            candidates.add(word_type(word))
    return candidates


In [4]:
vocab_OCC, sentences, tags, targets = read_in_data("./data/train")

In [5]:
vocab = remove_low_frequency_word(vocab_OCC, occurences = 2)

In [6]:
def get_idx_vocab_tags(vocab, tags):
    word_to_idx = {}
    tag_to_idx = {}
    # word_to_idx["unk"] = 1
    word_to_idx["PAD"] = 0
    start_word = 1
    for word in vocab:
        word_to_idx[word] = start_word
        start_word += 1
    start_tag = 0
    for tag in tags:
        tag_to_idx[tag] = start_tag
        start_tag += 1
    return word_to_idx, tag_to_idx

In [7]:
word_to_idx, tag_to_idx = get_idx_vocab_tags(vocab, tags)

In [8]:
len(word_to_idx)

10732

In [9]:
print(word_to_idx)

{'PAD': 0, 'Isaac': 1, 'coffee': 2, 'BSM': 3, 'loss': 4, 'somebody': 5, 'four-nation': 6, 'told': 7, 'Radio': 8, 'Sciences': 9, 'Kruger': 10, 'Just': 11, 'Tremila': 12, 'Avaz': 13, 'flashpoint': 14, 'S.AFRICAN': 15, 'Aviv': 16, 'Doorn': 17, 'NOTES': 18, 'information': 19, 'consumers': 20, 'Russo': 21, 'Bids': 22, 'Cosenza': 23, 'sequel': 24, 'Muslim': 25, 'Moxon': 26, 'Fleurian': 27, 'engineer': 28, 'Iran': 29, 'ninth': 30, 'Jr': 31, 'Jeremy': 32, 'believed': 33, 'allegedly': 34, 'vehicles': 35, 'unfair': 36, 'Norman': 37, 'funding': 38, 'Democracy': 39, 'father': 40, 'member': 41, 'Surabaya': 42, 'emphasis': 43, 'II': 44, 'delaying': 45, 'Bjorn': 46, 'vital': 47, 'elite': 48, 'off-spinner': 49, 'ingot': 50, 'intensive': 51, 'reviews': 52, 'Boland': 53, 'demonstration': 54, 'Samsung': 55, 'Ryder': 56, 'plantation': 57, 'causes': 58, 'FOURTH': 59, 'Kijimuta': 60, 'Va': 61, 'corporations': 62, 'contesting': 63, 'a.m.': 64, 'GMT': 65, 'lifted': 66, 'automation': 67, 'Gildea': 68, 'Almost'

In [10]:
def get_vocab_tags_idx(word_to_idx, tag_to_idx):
    idx_to_word, idx_to_tag = {}, {}
    for word, idx in word_to_idx.items():
        idx_to_word[idx] = word
    for tag, idx in tag_to_idx.items():
        idx_to_tag[idx] = tag
    return idx_to_word, idx_to_tag

In [11]:
idx_to_word, idx_to_tag = get_vocab_tags_idx(word_to_idx, tag_to_idx)

In [12]:
def process_sentences(sentences:list, word_to_idx:dict):
    data = []
    for sentence in sentences:
        tmp_sentence = []
        for word in sentence:
            if word in word_to_idx.keys():
                tmp_sentence.append(word_to_idx[word])
            else:
                tmp_sentence.append(word_to_idx[word_type(word)])
        data.append(tmp_sentence)
    return data

def process_targets(targets:list, tag_to_idx:dict):
    data = []
    for target in targets:
        tmp_target = []
        for tag in target:
            tmp_target.append(tag_to_idx[tag])
        data.append(tmp_target)
    return data
    

In [13]:
data_X = process_sentences(sentences, word_to_idx)

In [14]:
data_y = process_targets(targets, tag_to_idx)

In [15]:
# get the maximum length of sentence in train
maximum_length = 0
for sample in data_X:
    if len(sample) > maximum_length:
        maximum_length = len(sample)

In [16]:
maximum_length

113

In [17]:
def padding_sentence(data: list, maximum_length=125):
    for i in range(len(data)):
        if len(data[i]) > maximum_length:  # Truncating
            data[i] = data[i][:maximum_length]
        elif len(data[i]) < maximum_length:  # Padding 
            data[i] = data[i] + [0] * (maximum_length - len(data[i]))
    return data

def padding_tags(data: list, maximum_length=125):
    for i in range(len(data)):
        if len(data[i]) > maximum_length:  # Truncating
            data[i] = data[i][:maximum_length]
        elif len(data[i]) < maximum_length:  # Padding with penalty score
            data[i] = data[i] + [-100] * (maximum_length - len(data[i]))
    return data

In [18]:
data_X = padding_sentence(data_X)
data_y = padding_tags(data_y)

In [19]:
X_train = torch.LongTensor(data_X)
Y_train = torch.LongTensor(data_y)

In [20]:
print(X_train)

tensor([[ 507, 6994, 6259,  ...,    0,    0,    0],
        [4847, 1932,    0,  ...,    0,    0,    0],
        [6641, 1380,    0,  ...,    0,    0,    0],
        ...,
        [5494, 3067,    0,  ...,    0,    0,    0],
        [5518, 1807, 3175,  ...,    0,    0,    0],
        [7002,    0,    0,  ...,    0,    0,    0]])


In [21]:
ds_train = TensorDataset(X_train, Y_train)
loader_train = DataLoader(ds_train, batch_size=16, shuffle=False)

# model structure define

In [22]:
class BiLSTMNER(nn.Module):
    def __init__(self, vocab_size, target_size, embedding_dim=100, lstm_hidden_dim=256, lstm_layers=1, lstm_dropout=0.33, linear_dim=128):
        super().__init__()        
        self.dropout = nn.Dropout(0.33)
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, lstm_hidden_dim,num_layers=lstm_layers,batch_first=True, bidirectional=True) #dropout=lstm_dropout)
        self.linear = nn.Linear(lstm_hidden_dim * 2, linear_dim)
        self.elu = nn.ELU()
        self.classifier = nn.Linear(linear_dim, target_size)

    def forward(self, sentence):
        embeds = self.dropout(self.embedding(sentence))
        lstm_out,_ = self.lstm(embeds)
        lstm_out = self.dropout(lstm_out)
        linear_out = self.linear(lstm_out)
        elu_out = self.elu(linear_out)
        output = self.classifier(elu_out)
        return output

In [23]:
model = BiLSTMNER(vocab_size = len(word_to_idx), target_size = len(tag_to_idx))

In [24]:
def read_in_dev_data(filename):
    sentences = []
    tmp_sentence = []
    targets = []
    tmp_target = []
    with open(filename) as file:
        for line in file.readlines():
            if len(line) > 1:
                _, word, tag = line.strip().split(" ")
                tmp_sentence.append(word)
                tmp_target.append(tag)
            else:
                sentences.append(tmp_sentence)
                targets.append(tmp_target)
                tmp_sentence = []
                tmp_target = []
        if len(tmp_sentence)>0:
            sentences.append(tmp_sentence)
            targets.append(tmp_target)
    return sentences, targets

# dev dataset

In [25]:
dev_sentences, dev_targets = read_in_dev_data("./data/dev")

In [26]:
data_X_dev = process_sentences(dev_sentences, word_to_idx)
data_y_dev = process_targets(dev_targets, tag_to_idx)

In [27]:
data_X_dev = padding_sentence(data_X_dev)
data_y_dev = padding_tags(data_y_dev)

In [28]:
X_dev = torch.LongTensor(data_X_dev)
Y_dev = torch.LongTensor(data_y_dev)
ds_dev = TensorDataset(X_dev, Y_dev)
loader_dev = DataLoader(ds_dev, batch_size=16, shuffle=False)

# test dataset

In [29]:
def read_in_test_data(filename):
    sentences = []
    tmp_sentence = []
    with open(filename) as file:
        for line in file.readlines():
            if len(line) > 1:
                _, word = line.strip().split(" ")
                tmp_sentence.append(word)
            else:
                sentences.append(tmp_sentence)
                tmp_sentence = []
        if len(tmp_sentence)>0:
            sentences.append(tmp_sentence)
    return sentences

test_sentences = read_in_test_data("./data/test")

In [30]:
data_test = process_sentences(test_sentences, word_to_idx)

In [31]:
data_test = process_sentences(test_sentences, word_to_idx)
data_X_test = padding_sentence(data_test)
X_test = torch.LongTensor(data_X_test)
loader_test = DataLoader(X_test, batch_size=16, shuffle=False)

# Using GPU

In [32]:
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU mode")
else:
    device = torch.device("cpu")
    print("CPU mode")

GPU mode


# train and evaluate model

In [33]:
def train_evaluate(model, train_data, dev_data, epoch_num=25, tag_pad_idx = -100):
    optimizer = optim.SGD(model.parameters(), lr=0.23, momentum=0.9, nesterov=True)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=4)
    criterion = nn.CrossEntropyLoss(ignore_index= -100)
    best_loss = float('inf')
    predict_table = []
    for epoch in range(epoch_num):
        # training
        train_result = []
        train_loss = 0
        train_acc = 0
        train_total = 0
        model.train()
        for train_sentence, train_target in train_data:
            optimizer.zero_grad()
            train_sentence = train_sentence.to(device)
            train_target = train_target.to(device)
            train_pred = model(train_sentence)
            train_pred = train_pred.view(-1, train_pred.shape[-1])
            train_target = train_target.view(-1)
            train_tmp_loss = criterion(train_pred, train_target)

            train_tmp_total = 0
            train_tmp_correct = 0
            max_pred = train_pred.argmax(dim=1, keepdim = True)
            for tmp_pred, tmp_target, tmp_word in zip(max_pred, train_target, train_sentence.view(-1)):
                if tmp_word != 0:
                    train_result.append((tmp_word.item(), tmp_pred.item(), tmp_target.item()))
                    if tmp_target.item() == tmp_pred.item():
                        train_tmp_correct += 1
                    train_total += 1
            train_tmp_loss.backward()
            optimizer.step()
            train_loss += train_tmp_loss.item()
            train_acc += train_tmp_correct
            train_total += train_tmp_total
        print('Epoch ', epoch, ':')
        print(f'\tTrain Loss: {train_loss/len(train_data):.6f} | Train Acc: {(train_acc/train_total)*100:.2f}%')

        # evaluating
        dev_result = []
        dev_loss = 0
        dev_acc = 0
        dev_total = 0
        model.eval()
        with torch.no_grad():
            for dev_sentence, dev_target in dev_data:
                dev_sentence = dev_sentence.to(device)
                dev_target = dev_target.to(device)
                dev_pred = model(dev_sentence)
                dev_pred = dev_pred.view(-1, dev_pred.shape[-1])
                dev_target = dev_target.view(-1)
                dev_tmp_loss = criterion(dev_pred, dev_target)
                dev_tmp_total = 0
                dev_tmp_correct = 0
                max_pred = dev_pred.argmax(dim=1, keepdim=True)
                for tmp_pred, tmp_target, tmp_word in zip(max_pred, dev_target, dev_sentence.view(-1)):
                    if tmp_word != 0: # not padding word idx
                        dev_result.append((tmp_word.item(), tmp_pred.item(), tmp_target.item()))
                        if tmp_target.item() == tmp_pred.item():
                            dev_tmp_correct += 1
                        dev_total += 1
                dev_loss += dev_tmp_loss.item()
                dev_acc += dev_tmp_correct
                dev_total += dev_tmp_total
        print(f'\tDev Loss: {dev_loss / len(dev_data):.6f} | Dev Acc: {(dev_acc / dev_total) * 100:.2f}%')
        if dev_loss <= best_loss:
            best_loss = dev_loss
            predict_table = dev_result
            torch.save(model.state_dict(), './model/blstm1.pt')
    return predict_table

In [34]:
model.to(device)

BiLSTMNER(
  (dropout): Dropout(p=0.33, inplace=False)
  (embedding): Embedding(10732, 100, padding_idx=0)
  (lstm): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=128, bias=True)
  (elu): ELU(alpha=1.0)
  (classifier): Linear(in_features=128, out_features=9, bias=True)
)

In [35]:
out_pred = train_evaluate(model,loader_train,loader_dev)

Epoch  0 :
	Train Loss: 0.586061 | Train Acc: 85.66%
	Dev Loss: 0.400690 | Dev Acc: 88.78%
Epoch  1 :
	Train Loss: 0.401469 | Train Acc: 88.64%
	Dev Loss: 0.263702 | Dev Acc: 92.25%
Epoch  2 :
	Train Loss: 0.317175 | Train Acc: 90.47%
	Dev Loss: 0.207927 | Dev Acc: 93.92%
Epoch  3 :
	Train Loss: 0.273335 | Train Acc: 91.63%
	Dev Loss: 0.185394 | Dev Acc: 94.51%
Epoch  4 :
	Train Loss: 0.246520 | Train Acc: 92.20%
	Dev Loss: 0.170034 | Dev Acc: 94.93%
Epoch  5 :
	Train Loss: 0.219124 | Train Acc: 92.96%
	Dev Loss: 0.154983 | Dev Acc: 95.44%
Epoch  6 :
	Train Loss: 0.201589 | Train Acc: 93.45%
	Dev Loss: 0.151075 | Dev Acc: 95.45%
Epoch  7 :
	Train Loss: 0.188865 | Train Acc: 93.72%
	Dev Loss: 0.148224 | Dev Acc: 95.52%
Epoch  8 :
	Train Loss: 0.181581 | Train Acc: 94.00%
	Dev Loss: 0.137544 | Dev Acc: 95.88%
Epoch  9 :
	Train Loss: 0.165941 | Train Acc: 94.36%
	Dev Loss: 0.133823 | Dev Acc: 96.11%
Epoch  10 :
	Train Loss: 0.158357 | Train Acc: 94.64%
	Dev Loss: 0.132594 | Dev Acc: 96.09

In [36]:
y_pred = [int(x[1]) for x in out_pred]
print(len(y_pred))
length = 0
with open('./data/dev', "r") as file:
    for line in file:
        if len(line) > 1: 
            length+=1
file.close()
print(length)

51578
51578


In [37]:
i=0
newfile = open('./dev1.out', "w")
with open('./data/dev', "r") as file:
    for line in file:
        if len(line) > 1:
            idx, word, tag = line.strip().split(" ")
            newfile.write(str(idx)+' '+str(word)+' '+str(idx_to_tag[y_pred[i]])+'\n')
            i+=1
        else:
            newfile.write('\n')
file.close()
newfile.close()

# test predict

In [38]:
output = []
model.to(device)
model.eval()
for test_sentence in loader_test:
    test_sentence = test_sentence.to(device)
    test_pred = model(test_sentence)
    test_pred = test_pred.view(-1, test_pred.shape[-1])
    max_pred = test_pred.argmax(dim=1, keepdim = True)
    for tmp_pred, word in zip(max_pred, test_sentence.view(-1)):
        if word != 0:
            output.append(tmp_pred.item())

In [39]:
length = 0
with open('./data/test', "r") as file:
    for line in file:
        if len(line) > 1: 
            length+=1
file.close()
print(length)
print(len(output))

46666
46666


In [40]:
i=0
newfile = open('./test1.out', "w")
with open('./data/test', "r") as file:
    for line in file:
        if len(line) > 1:
            idx, word = line.strip().split(" ")
            newfile.write(str(idx)+' '+str(word)+' '+str(idx_to_tag[output[i]])+'\n')
            i+=1
        else:
            newfile.write('\n')
file.close()
newfile.close()